## 关于加工站‘垫刀’的机制建模及验证方式
___


本文由 [企鹅数据Penguin Statistics] @J 所撰写，@水晶泡芙工坊代发。

【 J: 手机验证不了，说不了话我要死了 QAQ | 就算你能说话了也无法独立完成发帖呀 Orz 】。

### 1. 引言
___
在 [https://www.bilibili.com/video/av56232063/] 视频中，UP主提到了“垫刀”的技巧，展示了用合成站刷高级材料的潜在可能。如果此结果为真，那么当前的很多优化工具很可能会失效（因为大部分算法没有考虑加工站的额外产出）。为了确保当前优化工具的合理性，我们为 “垫刀” 进行了建模，并用一个相对较小的样本进行分析。得到了 ”目前并未发现 ‘垫刀’ 机制存在” 的结论。



### 2. 什么是 ‘垫刀’ ？
___
为了考察 ‘垫刀’ 机制，我们需要给‘垫刀’做出数学定义。加工站‘垫刀’的通俗说法是 —— 给定前 $n$ 次加工没有触发加成，第 $n+1$ 加工更有可能触发加成。假定我们记事件 $A$ 为加工触发加成，事件 $B_n$ 为之前 $n$ 次加工均未触发加成，‘垫刀’即为 $P(A|B_n) \ge P(A)$, 即在 “之前 $n$ 次加工均为触发加成$(B_n)$的情况下，加工触发加成$(A)$” 的概率$P(A|B)$会高于普通的“加工触发加成$(A)$”的概率$P(A)$。

根据贝叶斯公式，我们有
$$
\begin{eqnarray*}
P(A|B) = \frac{P(A,B)}{P(B)} \overset{独立性}{=} \frac{P(A)P(B)}{p(B)} = P(A)
\end{eqnarray*}
$$
若独立性假设成立（"当次加工触发加成"的概率不受"之前的加工不触发加成"影响），我们可以得到有P$(A|B) = P(A)$。所以'垫刀'问题的本质是在讨论此独立性假设是否成立，于是我们得到以下建模。



### 3. 关于独立性的建模
___
我们记单次加工触发加成为$1$,不触发加成为$0$，即可得到一个伯努利随机变量$X_t \sim Bernoulli(p) \Leftrightarrow P( A ) = P(X_t=1) = p$,$p$为单次加工触发加成的概率。类似的，我们考虑‘存在垫刀机制下’的变量，我们记之前已有$n$次未触发加成 & 当次加工触发加成为$1$,不触发为$0$,  即可得到一个伯努利随机变量 $X_{t|n} \sim Bernoulli( p_n) \Leftrightarrow P(A | B_n ) = P( X_t =1| X_{t-1}= 0,...,X_{t-n} = 0) = p_n$, $n = 1,2,3...$. 为统一符号，我们令$p = p_0$,即前$0$次连续未触发加成，当次触发加成的概率。所以我们想要考察，是否存在 $p_0 = p_1 = p_2 = … =  p_n$.


### 4. 对于$p$的考察方式
___
频率学派即考察序列出现的频率。比如对于$p_0$,我们考察 ‘加成触发次数’ 占 ‘总加工次数’ 的比例； 对于 $n = 2$ 的情况则是考察'出现序列$[0,0,1]$的次数'占'出现序列$[0,0]$的次数'的比例 $\frac{ \#[0,0,1]}{ \#[0,0]}$ ，即 ‘两次未加后加成’ 占 ‘两次未加成的比例 （ '#' 代表计数）。
$$
p(A|B_2) = \frac{P(A,B_2)}{P(B_2)} = \frac{\#[0,0,1]/总次数}{\#[0,0]/总次数}= \frac{\#[0,0,1]}{\#[0,0]}
$$
对于$n = 3,4,...$ 的情况也可同样计算。



但由于个人不是很喜欢频率派做假设检验$(p_0 = p_1 = p_2 ,…)$时，大样本量，反复试验（中心极限定理）的解释/假设，所以我倾向贝叶斯派。我们使用一个 'Beta-Bernoulli' conjugate。假设我们有 Bernoulli($p$)-likelihood, Beta$(a,b)$-prior, 得到一个Beta的posterior -- Beta$( a + \sum_i x_i, b + m - \sum_i x_i)$. 此处的 $m$ 为总样本数，$\sum_i x_i$ 为其中触发加成的样本和。我们选用一个non-informative prior，令 $a = b = 0.001$。对应的贝叶斯检验就是考察随机变量 $\Delta_{i,j} = p_i - p_j$ 是否在95\%的 HPD (Highest Posterior Density) 区间内包含$0$.


此方法的好处是，由于估计对象 $p$ 被假定为随机变量（而不是频率学派的$p$是一个数）。$p$很自然的就有个对应的概率分布，我们就不用考虑CLT那一堆的大样本，反复试验的问题。


[一个简单的贝叶斯解释以及Beta-Bernoulli conjugate请参考http://www2.stat.duke.edu/~rcs46/modern_bayes17/lecturesModernBayes17/lecture-1/01-intro-to-Bayes.pdf ， J : 我永远迷信贝叶斯.jpg].



### 5. 基于小样本的实证
___

样本来源于 @水晶泡芙工坊，即本文代发者。我们在保持干员加成为75\%的水平下，记录了80次加成结果。

使用频率学派的方式，我们考察$n$次保底，$n=0,1,…9$ 对应上文的$ p_0, p_1, …, p_9$。汇报加成出现次数，总数，出货率（基于CLT的 $\frac{\sum_i x_i}{m}$) 和标准差 (基于CLT的 $\sqrt{ \frac{\sum_i x_i}{m}(1-\frac{\sum_i x_i}{m})/m}$)，结果如下
```{r}
              加成次数 总数  出货率  标准差
    0次保底        8   80  0.1000  0.0335
    1次保底        8   70  0.1143  0.0380
    2次保底        6   61  0.0984  0.0381
    3次保底        5   54  0.0926  0.0394
    4次保底        5   48  0.1042  0.0441
    5次保底        4   43  0.0930  0.0443
    6次保底        4   39  0.1026  0.0486
    7次保底        4   35  0.1143  0.0538
    8次保底        4   31  0.1290  0.0602
    9次保底        3   27  0.1111  0.0605
```

可以看出所以出货率的差值均在1个标准差之内。没有证据说明存在’垫刀‘的现象



使用贝叶斯学派的方式，我们汇报$\Delta_{i,j} = p_i - p_j$ 在 $95\%$ HPD区间的两端点，并检测是否区间包含$0$点。【注意到对角线均为$0$,这是因为此时对比的是这个分布于其本身，理所应当的$\Delta_{i,i} = 0$.】
```{r}
    # 左端点
         0     1     2     3     4     5     6     7     8     9
    0  0.00 -0.11 -0.10 -0.09 -0.11 -0.10 -0.12 -0.14 -0.16 -0.14
    1 -0.08  0.00 -0.09 -0.09 -0.11 -0.09 -0.11 -0.13 -0.16 -0.14
    2 -0.10 -0.12  0.00 -0.10 -0.12 -0.11 -0.13 -0.15 -0.18 -0.16
    3 -0.11 -0.13 -0.11  0.00 -0.13 -0.12 -0.14 -0.15 -0.18 -0.16
    4 -0.11 -0.12 -0.10 -0.10  0.00 -0.11 -0.12 -0.14 -0.17 -0.15
    5 -0.12 -0.13 -0.12 -0.11 -0.13  0.00 -0.15 -0.16 -0.19 -0.16
    6 -0.11 -0.13 -0.12 -0.11 -0.14 -0.12  0.00 -0.16 -0.18 -0.16
    7 -0.10 -0.12 -0.10 -0.11 -0.12 -0.11 -0.13  0.00 -0.18 -0.15
    8 -0.10 -0.11 -0.10 -0.10 -0.11 -0.11 -0.13 -0.14  0.00 -0.15
    9 -0.12 -0.13 -0.12 -0.11 -0.14 -0.12 -0.14 -0.16 -0.18  0.00
    
    # 右端点
         0    1    2    3    4    5    6    7    8    9
    0 0.00 0.08 0.10 0.11 0.11 0.12 0.11 0.10 0.10 0.12
    1 0.11 0.00 0.12 0.13 0.12 0.13 0.13 0.12 0.11 0.13
    2 0.10 0.09 0.00 0.11 0.10 0.12 0.12 0.10 0.10 0.12
    3 0.09 0.09 0.10 0.00 0.10 0.11 0.11 0.11 0.10 0.11
    4 0.11 0.11 0.12 0.13 0.00 0.13 0.14 0.12 0.11 0.14
    5 0.10 0.09 0.11 0.12 0.11 0.00 0.12 0.11 0.11 0.12
    6 0.12 0.11 0.13 0.14 0.12 0.15 0.00 0.13 0.13 0.14
    7 0.14 0.13 0.15 0.15 0.14 0.16 0.16 0.00 0.14 0.16
    8 0.16 0.16 0.18 0.18 0.17 0.19 0.18 0.18 0.00 0.18
    9 0.14 0.14 0.16 0.16 0.15 0.16 0.16 0.15 0.15 0.00
    
    # 是否包含0点
         0    1    2    3    4    5    6    7    8    9
    0 TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
    1 TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
    2 TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
    3 TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
    4 TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
    5 TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
    6 TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
    7 TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
    8 TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
    9 TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
```

可以看出所以区间都包含0点，说明贝叶斯方式的结论与频率学派方式的结论一致，并没有证据说明‘垫刀‘方式存在。

所以运算基于R，代码（附注释）与样本以附件形式上传。【 J : 我代码有点不太好，出了虫子请见谅Orz】



### 6. 总结
___

本文的目的在于提供一个思路和方式来考察‘垫刀‘这一现象是否存在，从而支持各种优化工具的可行性（如果‘垫刀’存在，应用于优化方法的合成公式矩阵以及定价方式可能都要根据 ‘用低成本素材垫刀 + 高等级素材加成 ’的刷素材方式重新计算）。【 J : 以及我们现在没有血亏！ www】

基于当前的数据，无论是频率学派方式还是贝叶斯派方式，我们都没有找到支持存在‘垫刀’现象的证据。当然，这并不能直接证明‘垫刀’这种现象不存在，我们依然在继续搜集数据来检验这一现象，而且希望数据越多越好。【正如我们并不能因为看到1000000只白鸽子，就说明其他颜色的鸽子不存在一样，我们不打算否定‘垫刀’存在的可能性。我们的思路更像是，【用大量样本都得不到支撑‘垫刀’的证据】来说明‘垫刀’不会对我们的加工产生太大的影响】。

还有就是，玄学出货有时候也挺令人开心的，各位多科塔相信自己想相信的事情，玩得开心就好 www


